In [1]:
!pip install datasets
!pip install speechbrain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.2/807.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 20.8 MB/s eta 0:00:00


## Load the model

In [2]:
from transformers import AutoProcessor, AutoModelForTextToSpectrogram, SpeechT5HifiGan

# Load English model
processor = AutoProcessor.from_pretrained("Tejasva-Maurya/English_Technical_finetuned")
model = AutoModelForTextToSpectrogram.from_pretrained("Tejasva-Maurya/English_Technical_finetuned")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

## Creating Embeddings

In [3]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier
from datasets import load_dataset, Audio

dataset = load_dataset("csv", data_files = "/content/drive/MyDrive/Colab Notebooks/TTS/ALL_DATASET/metadata_COLAB.csv",split="train", trust_remote_code=True)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)
def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings
def prepare_dataset(example):
    audio = example["audio"]
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])
    return example


# Calculate the number of rows for a part of the dataset
part = len(dataset) //1000

# Select the part of the dataset
dataset = dataset.select(range(part))

# Prepare the dataset
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)
example = dataset[10]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

<ipython-input-3-6a86a0c4f9df>:3: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier


Generating train split: 0 examples [00:00, ? examples/s]

hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

## Text Preprocessing

In [4]:
import re
def text_preprocessing(text):
     replacements = [
     ("0", "zero"),
     ("1", "one"),
     ("2", "two"),
     ("3", "three"),
     ("4", "four"),
     ("5", "five"),
     ("6", "six"),
     ("7", "seven"),
     ("8", "eight"),
     ("9", "nine"),
     ("_", " ")]
      # Convert to lowercase
     text = text.lower()

     # Remove punctuation (except apostrophes)
     text = re.sub(r'[^\w\s\']', '', text)

     # Remove extra whitespace
     text = ' '.join(text.split())
     for src, dst in replacements:
       text = text.replace(src, dst)
     return text

## Audio Generation

In [7]:
text ="CUDA is a parallel computing platform and programming model."
text = text_preprocessing(text)
inputs = processor(text=text, return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [8]:
from IPython.display import Audio
Audio(speech.numpy(), rate = 16000)